In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta
import zenhan
from jeraconv import jeraconv
j2w = jeraconv.J2W()

import subprocess
import os

In [2]:
FILE_DATE = "20220705"
FILE_DATE2 = "20220518"
FILE_LIST = [    
    # 令和2年
    "./rawdata/4_teian_kisei_R2_%s.xlsx"%FILE_DATE,
    "./rawdata/8_teian_gyoukaku_R2_%s.xlsx"%FILE_DATE,
    "./rawdata/12_kaitou_kisei_R2_%s.xlsx"%FILE_DATE,
    "./rawdata/16_kaitou_gyoukaku_R2_%s.xlsx"%FILE_DATE,
    # 令和3年
    "./rawdata/2_teian_kisei_R3_%s.xlsx"%FILE_DATE,
    "./rawdata/6_teian_gyoukaku_R3_%s.xlsx"%FILE_DATE,
    "./rawdata/10_kaitou_kisei_R3_%s.xlsx"%FILE_DATE,
    "./rawdata/14_kaitou_gyoukaku_R3_%s.xlsx"%FILE_DATE
]
JSON_SUFFIX = "_processed" 

In [3]:
baseurl = "https://www.gyoukaku.go.jp/hotline/img/"
for filepath in FILE_LIST:
    if not os.path.exists(filepath):
        url = baseurl + os.path.basename(filepath)
        subprocess.call(['wget', '--directory-prefix', './rawdata', url], shell=True)

## Helper関数

In [4]:
import datetime

## Cases
# STRING: '令和\n3年'
# INT: EXCEL DATE
# DATETIME: datetime

def normalize(data):
    if isinstance(data, int) or isinstance(data, float):
        return pd.to_datetime('1900-01-01') + pd.to_timedelta(data, 'D')
    elif isinstance(data, datetime.datetime):
        return data
    elif isinstance(data, str):
        oneline = zenhan.z2h(data.replace('\n', ''))
        Y, MD = oneline.split('年')
        try:
            Y = str(j2w.convert(Y + '年')) + '年'
        except:
            Y = str(j2w.convert('令和' + Y + '年')) + '年'
        date = datetime.datetime.strptime(Y+MD, '%Y年%m月%d日')
        return date
    else:
        return data

def conv_date(df, column):
    tmp = df[column].apply(normalize)
    return tmp

## 本題

In [5]:
df_list = []
for k, y, h, i in zip(
				["規制改革", "行政改革", "規制改革", "行政改革"], 
				[2, 2, 3, 3],
				# header
				[3, 2, 3, 1],
				# Order in list
				[0, 1, 4, 5]):
	print(FILE_LIST[i])
	df = pd.read_excel(FILE_LIST[i], header=h)
	df.rename(columns={
		'所管省庁への　　検討要請日': '所管省庁への検討要請日',
		'所管省庁への\n検討要請日': '所管省庁への検討要請日',
		'所管省庁\nへの検討\n要請日': '所管省庁への検討要請日',
		'所管省庁\nへの検討要請日': '所管省庁への検討要請日',
		'内閣府での\n回答取りまとめ\n日':'回答取りまとめ日',
		'内閣府での\n回答取りまとめ日':'回答取りまとめ日',
		'提案事項名\n（タイトル）':'提案事項',
		'提案主体名\n（会社名・団体名）':'提案主体',
		'制度の所管省庁': '所管省庁',
		'ワーキング・グループにおける\n処理方針': 'ワーキング・グループにおける処理方針'
	}, inplace=True)
	print(df.columns)
	df['所管省庁への検討要請日'] = conv_date(df, '所管省庁への検討要請日')
	df['回答取りまとめ日'] = conv_date(df, '回答取りまとめ日')

	print(FILE_LIST[i+2])
	df2 = pd.read_excel(FILE_LIST[i+2], header=[0, 1])
	df2.rename(columns={
		'所管省庁への\n検討要請日': '所管省庁への検討要請日',
		'所管省庁\nへの検討\n要請日': '所管省庁への検討要請日',
		'所管省庁\nへの検討要請日': '所管省庁への検討要請日',
		'内閣府での\n回答取りまとめ\n日':'回答取りまとめ日',
		'提案事項名\n（タイトル）':'提案事項名',
		'提案主体名\n（会社名・団体名）':'提案主体名',
		'ワーキング・グループにおける\n処理方針': 'ワーキング・グループにおける処理方針'
	}, inplace=True)
	df2.columns = ["_".join(x) if isinstance(x[1], str) and x[1][0:3] != "Unn" else x[0].replace('\n','') for x in df2.columns.ravel()]
	df2['所管省庁への検討要請日'] = conv_date(df2, '所管省庁への検討要請日')
	df2['回答取りまとめ日'] = conv_date(df2, '回答取りまとめ日')

	df3 = pd.merge(df, df2, how='left', on='番号')
	df3.drop(
		columns=[
			'提案事項_y', '提案主体_y', '所管省庁_y', '備考', '備考_x', '備考_y', 'Unnamed: 7',
			'ワーキング・グループにおける処理方針', 'ワーキング・グループにおける処理方針_x', 'ワーキング・グループにおける処理方針_y'],
		inplace = True, errors='ignore')
	def a(x):
		try:
			tmp = x['所管省庁への検討要請日_y']
			if tmp is pd.NaT:
				tmp = x['所管省庁への検討要請日_x']
		except:
			print(x)
		return tmp
	df3['所管省庁への検討要請日_x'] = df3.apply(a, raw=False, axis=1)

	def b(x):
		try:
			tmp = x['回答取りまとめ日_x']
			if tmp == pd.NaT:
				tmp = x['回答取りまとめ日_y']
		except:
			print(x)
		return tmp
	df3['回答取りまとめ日_x'] = df3.apply(b, raw=False, axis=1)
	df3= df3.drop(columns=['所管省庁への検討要請日_y', '回答取りまとめ日_y'])

	df3.rename(columns={
		'所管省庁への検討要請日_x': '所管省庁への検討要請日',
		'回答取りまとめ日_x': '回答取りまとめ日',
		'提案事項_x': '提案事項',
		'提案主体_x': '提案主体',
		'所管省庁_x': '所管省庁',
		'所管省庁の検討結果_対応の\n分類': '所管省庁の検討結果_対応の分類'
	}
	, inplace=True)

	df3.dropna(subset=['番号'], inplace=True)
	df3['番号'] = df3['番号'].astype(int)
	df3['区分'] = k
	df3['通番'] = k[0] + str(y) + df3['番号'].map('-{:04d}'.format)
	print(df3.columns)

	df_list.append(df3.copy())

df_all = pd.concat(df_list, ignore_index=True)

./rawdata/4_teian_kisei_R2_20220705.xlsx
Index(['番号', '所管省庁への検討要請日', '回答取りまとめ日', '提案事項', '提案主体', '所管省庁', '備考',
       'ワーキング・グループにおける処理方針'],
      dtype='object')
./rawdata/12_kaitou_kisei_R2_20220705.xlsx


c:\Users\tsasa\miniconda3\lib\site-packages\ipykernel_launcher.py:38: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.


Index(['番号', '所管省庁への検討要請日', '回答取りまとめ日', '提案事項', '提案主体', '所管省庁', '提案の具体的内容',
       '提案理由', '所管省庁の検討結果_制度の現状', '所管省庁の検討結果_該当法令等', '所管省庁の検討結果_対応の分類',
       '所管省庁の検討結果_対応の概要', '区分', '通番'],
      dtype='object')
./rawdata/8_teian_gyoukaku_R2_20220705.xlsx
Index(['番号', '所管省庁への検討要請日', '回答取りまとめ日', '提案事項', '提案主体', '所管省庁', '備考'], dtype='object')
./rawdata/16_kaitou_gyoukaku_R2_20220705.xlsx


c:\Users\tsasa\miniconda3\lib\site-packages\ipykernel_launcher.py:38: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.


Index(['番号', '所管省庁への検討要請日', '回答取りまとめ日', '提案事項', '提案主体', '所管省庁', '提案の具体的内容',
       '提案理由', '所管省庁の検討結果_制度の現状', '所管省庁の検討結果_該当法令等', '所管省庁の検討結果_対応の分類',
       '所管省庁の検討結果_対応の概要', '区分', '通番'],
      dtype='object')
./rawdata/2_teian_kisei_R3_20220705.xlsx
Index(['番号', '所管省庁への検討要請日', '回答取りまとめ日', '提案事項', '提案主体', '所管省庁', '備考',
       'ワーキング・グループにおける処理方針'],
      dtype='object')
./rawdata/10_kaitou_kisei_R3_20220705.xlsx


c:\Users\tsasa\miniconda3\lib\site-packages\ipykernel_launcher.py:38: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.


Index(['番号', '所管省庁への検討要請日', '回答取りまとめ日', '提案事項', '提案主体', '所管省庁', '提案の具体的内容',
       '提案理由', '所管省庁の検討結果_制度の現状', '所管省庁の検討結果_該当法令等', '所管省庁の検討結果_対応の分類',
       '所管省庁の検討結果_対応の概要', '区分', '通番'],
      dtype='object')
./rawdata/6_teian_gyoukaku_R3_20220705.xlsx
Index(['番号', '所管省庁への検討要請日', '回答取りまとめ日', '提案事項', '提案主体', '所管省庁', '備考'], dtype='object')
./rawdata/14_kaitou_gyoukaku_R3_20220705.xlsx
Index(['番号', '所管省庁への検討要請日', '回答取りまとめ日', '提案事項', '提案主体', '所管省庁', '提案の具体的内容',
       '提案理由', '所管省庁の検討結果_制度の現状', '所管省庁の検討結果_該当法令等', '所管省庁の検討結果_対応の分類',
       '所管省庁の検討結果_対応の概要', '区分', '通番'],
      dtype='object')


c:\Users\tsasa\miniconda3\lib\site-packages\ipykernel_launcher.py:38: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.


In [6]:
df_all = df_all.loc[:, ~ df_all.columns.str.contains('^Unnamed')]

In [7]:
df_all

,番号,所管省庁への検討要請日,回答取りまとめ日,提案事項,提案主体,所管省庁,提案の具体的内容,提案理由,所管省庁の検討結果_制度の現状,所管省庁の検討結果_該当法令等,所管省庁の検討結果_対応の分類,所管省庁の検討結果_対応の概要,区分,通番
0,1,2020-10-21,2020-11-26,技術士（農業部門）及び技術士補（農業部門）登録者の（農業）普及指導員への任用について,個人,農林水産省,\n平成16年度まで普及職員の資格試験は「改良普及員」と「専門技術員」の２つが実施されていま...,\n普及指導員は高度な普及事業を担当する即戦力の技術者として任用されることから「普及指導員資...,\n普及指導員は、都道府県職員の役職であり、都道府県知事が任用するものです。このため、原則普...,\n農業改良助長法施行令（昭和27年政令第148号）第３条第２号、農業改良助長法施行令に規定...,\n現行制度下で対応可能\n,\n「農業改良助長法施行令（昭和27年政令第148号）第３条第２号」及び「農業改良助長法施行...,規制改革,規2-0001
1,2,2020-10-21,2020-11-11,法務局の登記簿謄本と公図のデジタル化,個人,法務省,\n現在、不動産の登記簿謄本や商業謄本（会社謄本）は紙データ（ＰＤＦデータ）で法務局から受け...,\n現在、不動産の登記簿謄本や商業謄本（会社謄本）は紙データ（ＰＤＦデータ）で法務局から受け...,\n不動産登記手続及び商業・法人登記手続においては，登記事項証明書，地図，建物所在図，地図に...,\n不動産登記法第１１９条及び１２０条，商業登記法第１０条，電気通信回線による登記情報の提供...,\n対応不可\n,\n登記制度は，登記記録に記録された内容を公示することにより，国民の権利の保全や商号，会社等...,規制改革,規2-0002
2,3,2020-10-21,2020-11-26,保険について,個人,厚生労働省,\nコロナ禍の中ご苦労様です。私茅ヶ崎市在住の者ですが、社会保険加入と国民健康保険の切り替え...,\n今後のデジタル化に伴いマイナンバーカードの利点をアナログからデジタル化活用し無駄な動きが...,\n 社会保険加入について、国保保険者に自動的に伝える仕組みについては、現時点では、社会保険...,\n\n,\n対応不可\n,\n制度の現状欄に記載のとおりです。\n,規制改革,規2-0003
3,4,2020-10-21,NaT,一般ゴミ規制について,個人,環境省,NaN,NaN,NaN,NaN,NaN,NaN,規制改革,規2-0004
4,5,2020-10-21,2020-11-11,成人向け作品におけるモザイクや消しについて,個人,警察庁\n法務省,\n成人向けの性的な描写におけるモザイク等の修正が必要とされている現状の無意味な規制を撤廃す...,\n現在成人向けコンテンツにおける性器に対して掛けられている修正は世界的に見てもその必要性が...,\n刑法第175条は、「成人向けの性的な描写におけるモザイク等の修正が必要」という規制を規定...,\nなし\n,\n事実誤認\n,\n制度の現状欄に記載のとおり「規制」についての事実誤認があります。\nなお、捜査機関におい...,規制改革,規2-0005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3656,258,2022-06-05,2022-06-29,独立行政法人のＤＸ,個人,デジタル庁\n総務省,民間企業同士のやりとりと同じレベルまで独立行政法人もデジタルトランスフォーメーションに取り組む。,\n企業同士での契約は電子契約が主流になりつつあるのに、独立行政法人との契約では、契約書や納...,「デジタル社会の実現に向けた重点計画」（令和４年６月７日閣議決定）第６ ５（１）⑬に定めると...,なし,検討を予定,「デジタル社会の実現に向けた重点計画」（令和４年６月７日閣議決定）において、デジタル庁は、独...,行政改革,行3-0258
3657,259,2022-06-05,NaT,ICD-11の日本国内への早期適用(約3年以上要している審査プロセスの迅速化),個人,厚生労働省\n総務省,NaN,NaN,NaN,NaN,NaN,NaN,行政改革,行3-0259
3658,260,2022-06-05,2022-06-29,代理人の過誤による更正登記の申請に、本人の委任状を不要とすること,商業登記ゲンロン,法務省,\n登記官による錯誤又は遺漏は登記官が職権で登記を更正し、申請人による場合は、申請人が登録免...,\n理由は、次のとおりである。\n1.まず、政府の統一方針であるデジタル・ガバメント実行計画...,代理人の過失等によって登記と実体関係との間に不一致が生じ、更正の登記を行うこととなった場合...,不動産登記令第７条第１項第２号,対応不可,登記は、原則として、当事者の申請等がなければすることができないとされており、代理人によって...,行政改革,行3-0260
3659,261,2022-06-05,2022-06-29,国税庁の法人番号公表サイトに掲載されている法人について、商業登記の有無を公表すること,個人,財務省\n法務省,国税庁の法人番号公表サイトに掲載されている法人について、商業登記の有無を公示することで、法人...,\n現在、国税庁の法人番号公表サイトには商業登記がある法人と登記されていない法人が混在してい...,国税庁法人番号公表サイト（以下「公表サイト」といいます。）では、法務省から提供される登記情...,・行政手続における特定の個人を識別するための番号の利用等に関する法律\n・行政手続における特...,対応,「制度の現状」欄に記載のとおりです。,行政改革,行3-0261


## タグ付処理

In [8]:
import MeCab
tagger = MeCab.Tagger("-Ochasen -u ./dict/local_dict.dic")

In [9]:
def get_word_bag(text):
    try:
        tag = tagger.parseToNode(text)
        words = set()
        while tag:
            feature = tag.feature.split(",")
            if feature[0] == "名詞" and len(tag.surface) >= 2:
                words.add(tag.surface)
            tag = tag.next
        return list(words)
    except:
        return []

In [10]:
all_text = df_all['提案事項'] + \
           df_all['提案の具体的内容'].replace(np.nan, '', regex=True) + \
           df_all['提案理由'].replace(np.nan, '', regex=True) + \
           df_all['所管省庁の検討結果_制度の現状'].replace(np.nan, '', regex=True) + \
           df_all['所管省庁の検討結果_対応の概要'].replace(np.nan, '', regex=True) + \
           df_all['所管省庁の検討結果_該当法令等'].replace(np.nan, '', regex=True)
tmp = all_text.apply(get_word_bag)

In [11]:
import json

with open('./dict/tag.json', encoding='utf-8') as json_file:
    rules_json = json.load(json_file)

def get_summary_tag(words):
    tags = []
    rules = rules_json['rules']

    for rule in rules:
        if any(item in words for item in rule['def']):
            tags.append(rule['tag'])
    return '\n'.join(map(str,tags))

In [12]:
df_all['タグ'] = tmp.apply(get_summary_tag)

## 過去ログ参照

In [13]:
df_all.loc[df_all['所管省庁の検討結果_制度の現状'].str.match('^(番号|規制改革|行政改革)', na=False), '所管省庁の検討結果_対応の分類'] = '回答済'

## データ保存

In [14]:
# 概要
df_all['UID'] = df_all.index.map('UID{:05d}'.format)
df_all.drop_duplicates(subset=['通番'], inplace=True)
df_all[['UID','番号', '回答取りまとめ日', '提案事項', '提案主体', '所管省庁', '所管省庁への検討要請日','所管省庁の検討結果_対応の分類',
        '区分', '通番','タグ']].to_json('./s3static/src/assets/data{}.json'.format(JSON_SUFFIX), orient='records', force_ascii=False)
# 詳細
df_detailed = df_all[['UID','提案の具体的内容', '提案理由', '所管省庁の検討結果_制度の現状', '所管省庁の検討結果_該当法令等',
       '所管省庁の検討結果_対応の分類', '所管省庁の検討結果_対応の概要']]
df_detailed = df_detailed.set_index('UID')
df_detailed.dropna(subset=['提案の具体的内容'], inplace=True) 
df_detailed.to_json('./s3static/public/data/data{}.json'.format(JSON_SUFFIX), orient='index', force_ascii=False)

In [15]:
df_all.columns

Index(['番号', '所管省庁への検討要請日', '回答取りまとめ日', '提案事項', '提案主体', '所管省庁', '提案の具体的内容',
       '提案理由', '所管省庁の検討結果_制度の現状', '所管省庁の検討結果_該当法令等', '所管省庁の検討結果_対応の分類',
       '所管省庁の検討結果_対応の概要', '区分', '通番', 'タグ', 'UID'],
      dtype='object')

In [16]:
df_all

,番号,所管省庁への検討要請日,回答取りまとめ日,提案事項,提案主体,所管省庁,提案の具体的内容,提案理由,所管省庁の検討結果_制度の現状,所管省庁の検討結果_該当法令等,所管省庁の検討結果_対応の分類,所管省庁の検討結果_対応の概要,区分,通番,タグ,UID
0,1,2020-10-21,2020-11-26,技術士（農業部門）及び技術士補（農業部門）登録者の（農業）普及指導員への任用について,個人,農林水産省,\n平成16年度まで普及職員の資格試験は「改良普及員」と「専門技術員」の２つが実施されていま...,\n普及指導員は高度な普及事業を担当する即戦力の技術者として任用されることから「普及指導員資...,\n普及指導員は、都道府県職員の役職であり、都道府県知事が任用するものです。このため、原則普...,\n農業改良助長法施行令（昭和27年政令第148号）第３条第２号、農業改良助長法施行令に規定...,\n現行制度下で対応可能\n,\n「農業改良助長法施行令（昭和27年政令第148号）第３条第２号」及び「農業改良助長法施行...,規制改革,規2-0001,教育\n一元化,UID00000
1,2,2020-10-21,2020-11-11,法務局の登記簿謄本と公図のデジタル化,個人,法務省,\n現在、不動産の登記簿謄本や商業謄本（会社謄本）は紙データ（ＰＤＦデータ）で法務局から受け...,\n現在、不動産の登記簿謄本や商業謄本（会社謄本）は紙データ（ＰＤＦデータ）で法務局から受け...,\n不動産登記手続及び商業・法人登記手続においては，登記事項証明書，地図，建物所在図，地図に...,\n不動産登記法第１１９条及び１２０条，商業登記法第１０条，電気通信回線による登記情報の提供...,\n対応不可\n,\n登記制度は，登記記録に記録された内容を公示することにより，国民の権利の保全や商号，会社等...,規制改革,規2-0002,デジタル\nハンコ,UID00001
2,3,2020-10-21,2020-11-26,保険について,個人,厚生労働省,\nコロナ禍の中ご苦労様です。私茅ヶ崎市在住の者ですが、社会保険加入と国民健康保険の切り替え...,\n今後のデジタル化に伴いマイナンバーカードの利点をアナログからデジタル化活用し無駄な動きが...,\n 社会保険加入について、国保保険者に自動的に伝える仕組みについては、現時点では、社会保険...,\n\n,\n対応不可\n,\n制度の現状欄に記載のとおりです。\n,規制改革,規2-0003,マイナンバーカード\nデジタル\n医療\nコロナ,UID00002
3,4,2020-10-21,NaT,一般ゴミ規制について,個人,環境省,NaN,NaN,NaN,NaN,NaN,NaN,規制改革,規2-0004,,UID00003
4,5,2020-10-21,2020-11-11,成人向け作品におけるモザイクや消しについて,個人,警察庁\n法務省,\n成人向けの性的な描写におけるモザイク等の修正が必要とされている現状の無意味な規制を撤廃す...,\n現在成人向けコンテンツにおける性器に対して掛けられている修正は世界的に見てもその必要性が...,\n刑法第175条は、「成人向けの性的な描写におけるモザイク等の修正が必要」という規制を規定...,\nなし\n,\n事実誤認\n,\n制度の現状欄に記載のとおり「規制」についての事実誤認があります。\nなお、捜査機関におい...,規制改革,規2-0005,海外,UID00004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3656,258,2022-06-05,2022-06-29,独立行政法人のＤＸ,個人,デジタル庁\n総務省,民間企業同士のやりとりと同じレベルまで独立行政法人もデジタルトランスフォーメーションに取り組む。,\n企業同士での契約は電子契約が主流になりつつあるのに、独立行政法人との契約では、契約書や納...,「デジタル社会の実現に向けた重点計画」（令和４年６月７日閣議決定）第６ ５（１）⑬に定めると...,なし,検討を予定,「デジタル社会の実現に向けた重点計画」（令和４年６月７日閣議決定）において、デジタル庁は、独...,行政改革,行3-0258,デジタル\nハンコ,UID03656
3657,259,2022-06-05,NaT,ICD-11の日本国内への早期適用(約3年以上要している審査プロセスの迅速化),個人,厚生労働省\n総務省,NaN,NaN,NaN,NaN,NaN,NaN,行政改革,行3-0259,,UID03657
3658,260,2022-06-05,2022-06-29,代理人の過誤による更正登記の申請に、本人の委任状を不要とすること,商業登記ゲンロン,法務省,\n登記官による錯誤又は遺漏は登記官が職権で登記を更正し、申請人による場合は、申請人が登録免...,\n理由は、次のとおりである。\n1.まず、政府の統一方針であるデジタル・ガバメント実行計画...,代理人の過失等によって登記と実体関係との間に不一致が生じ、更正の登記を行うこととなった場合...,不動産登記令第７条第１項第２号,対応不可,登記は、原則として、当事者の申請等がなければすることができないとされており、代理人によって...,行政改革,行3-0260,デジタル,UID03658
3659,261,2022-06-05,2022-06-29,国税庁の法人番号公表サイトに掲載されている法人について、商業登記の有無を公表すること,個人,財務省\n法務省,国税庁の法人番号公表サイトに掲載されている法人について、商業登記の有無を公示することで、法人...,\n現在、国税庁の法人番号公表サイトには商業登記がある法人と登記されていない法人が混在してい...,国税庁法人番号公表サイト（以下「公表サイト」といいます。）では、法務省から提供される登記情...,・行政手続における特定の個人を識別するための番号の利用等に関する法律\n・行政手続における特...,対応,「制度の現状」欄に記載のとおりです。,行政改革,行3-0261,デジタル,UID03659
